# fea try
* do the hw3from the fea course I took a while ago
* but try it in julia

In [1]:
# import Pkg; Pkg.add("StaticArrays")

In [2]:
using Plots
using StaticArrays
include("3_fea_helper.jl")

basis_gradient (generic function with 1 method)

In [42]:
Emod = 2e11
nu = 0.3
h3 = 1e9
L = 0.1

basisFunctionOrder=1;

In [39]:
# each row is x,y,z
# column 1 is the min, column 2 is the max
domain_size = Array{Float32}(undef,3,2)
domain_size[1,:] = [0,1]; # x dimensions
domain_size[2,:] = [0,1]; # y dimensions
domain_size[3,:] = [0,1]; # z dimensions

elm_per_dim=3  # elements per dimension
nodes_per_dim = elm_per_dim+1


num_elm = elm_per_dim^3
num_nodes = num_elm+1

28

# define the node locations

In [5]:
struct Node
    id::Int
    x::Float32
    y::Float32
    z::Float32
end

In [6]:
nodes = Array{Node}(undef, nodes_per_dim^3);
temp_locs = collect(range(0,stop=1,length = nodes_per_dim))
id = 1
for z in temp_locs
   for y in temp_locs
       for x in temp_locs
#             println("$id $x $y $z")
            mynode = Node(id,x,y,z)
            nodes[id] = mynode
            id+=1
        end        
    end
end

# Define the elements

In [7]:
# https://discourse.julialang.org/t/how-to-place-an-array-as-one-of-the-fields-of-a-mutable-struct/16939/3
struct Elem
   id::Int32
   node_ids::MVector{8, Int32}
    
end



In [8]:
elements = Array{Elem}(undef, elm_per_dim^3);
temp_locs = collect(range(0,stop=elm_per_dim-1))
nd = nodes_per_dim # abrieviate to nd
id = 1
for z in temp_locs
   for y in temp_locs
       for x in temp_locs
#             println("$id $x $y $z")
#             x,y,z is the position of the bottom front left corner
            s = x+y*nd+z*nd^2 # s = start
            # follow the abaqus node numbering for a hex
            node_ids = [s,s+1,s+nd^2+1, s+nd^2, s+nd,s+nd+1, s+nd^2+nd+1, s+nd^2+nd]
            node_ids.+=1
            myelem = Elem(id,node_ids)
            elements[id] = myelem
            id+=1
        end        
    end
end

# Define a quad rule

In [9]:
struct  Quad
   quad_points # = [-sqrt(1/3), sqrt(1/3)],
   quad_weights# = [1, 1]    
end

In [10]:
points = [-sqrt(1/3), sqrt(1/3)]
weights = [1,1]
myquad =  Quad(points, weights)

Quad([-0.5773502691896257, 0.5773502691896257], [1, 1])

# Define elasticity tensor

In [11]:
function C(i::Int,j::Int, k::Int,l::Int)
    #   //Define the material parameters of Young's modulus and Poisson's ratio
    E = Emod
    lambda = (E * nu) / ((1 + nu) * (1 - 2 * nu));
     mu = E / (2. * (1. + nu));

  return lambda * (i == j) * (k == l) + mu * ((i == k) * (j == l) + (i == l) * (j == k));
end

C (generic function with 1 method)

In [12]:
C(1,1,1,1)

2.6923076923076923e11

In [13]:
# elements

In [14]:
4^3

64

In [15]:
# ?range

In [16]:

# g1 = 0
# g2 = 0.001

# u_0 = g1
# u_l = g2

# f_bar =10e11

# function f(x)
#    return x*f_bar 
# end
# num_elm = 100

# h = 10e6
# dofs_per_elem = 2
# # dofs_per_elem_values = collect(1:dofs_per_elem)
# quad_rule = 2
# quad_rule_values = collect(1:quad_rule)
# basisFunctionOrder  = 1
# area = 10e-4
# modulus = 10e11

struct  Quad
   quad_points # = [-sqrt(1/3), sqrt(1/3)],
   quad_weights# = [1, 1]
    
end



In [17]:
# quad_rule_values

In [18]:
points = [-sqrt(1/3), sqrt(1/3)]
weights = [1,1]
myquad =  Quad(points, weights)

Quad([-0.5773502691896257, 0.5773502691896257], [1, 1])

In [43]:

step = L/num_elm
node_loc =  collect(range(0, length=num_nodes, stop=L))

# elem_node_map  = zeros(Int8,num_elm,2); # Array{Float64, 2}(undef, 2, 3)


# # elem_node_map  = zeros(Int8,num_elm,2); # Array{Float64, 2}(undef, 2, 3)
# for i in collect(1:num_elm)
#    elem_node_map[i,:]=[i,i+1] 
# #     println(i)
# #      setindex!(elem_node_map,
# end

# # elem_node_map

# function get_dofs_for_elem(elm_num)
#    # get th`ze  global dofs for this element
#     return elem_node_map[elm_num,:]
    
# end

# get_dofs_for_elem(5)

# Boundary conditions

In [44]:
# setup a vector representing the Dirichlet Boundary Conditions (the fixed BCs)
# boundary_values =  Dict()# zeros(Float32,num_nodes );
disp = zeros(Float32, num_nodes)
free_nodes = Array{Bool}(undef,num_nodes)
fill!(free_nodes, true)
# println(free_nodes)
for node_idx  in 1:num_nodes
    if node_loc[node_idx] ==0
        disp[node_idx] = u_0
        free_nodes[node_idx] = false
    elseif node_loc[node_idx] ==L
         disp[node_idx] = u_l
        free_nodes[node_idx] = false        
    end
end
# all_nodes;

LoadError: UndefVarError: u_0 not defined

In [21]:
# I'm sure there is a better way to do this, but not sure of it right now. 
# all_nodes = collect(1:num_nodes)
# fixed_nodes = keys(boundary_values)
# free_nodes = [i for i in all_nodes if !(i in fixed_nodes)];
# free_nodes_bool = [!(i in fixed_nodes) for i in all_nodes  ];
fixed_nodes = .!free_nodes;

LoadError: UndefVarError: free_nodes not defined

In [22]:
Fglobal = zeros(Float32,num_nodes );
Kglobal = zeros(Float32, num_nodes, num_nodes);

LoadError: UndefVarError: num_nodes not defined

In [23]:
Flocal = zeros(Float32, dofs_per_elem)
Klocal = zeros(Float32,dofs_per_elem,dofs_per_elem)
for i in collect(1:num_elm)
    # zero out previous values. 
    fill!(Flocal, 0)
    fill!(Klocal,0)
    
    # loop over the elements    
    elem_dofs = get_dofs_for_elem(i)
    h_e  = node_loc[elem_dofs[2]] - node_loc[elem_dofs[1]]
    # ---------------------------    
    # calculate the local  forcing vector
    # ---------------------------
    temp_f_constants = area*h_e/2       
    for A in 1:dofs_per_elem
#         F_value_at_quad_point=0;
        for q in quad_rule_values
            x = 0
           for B in 1:dofs_per_elem
                x += node_loc[elem_dofs[B]]*basis_function(B,myquad.quad_points[q])
#                 println("generating local f. A = $A, q = $q, B = $B, x_position = $x")                
            end
            F_value_at_quad_point = f(x);
            n_at_quad_point = basis_function(A,q);
            Flocal[A] += F_value_at_quad_point*n_at_quad_point*temp_f_constants*myquad.quad_weights[q];  
        end
    end    
    
    # ---------------------------
    # Generate K local
    # ---------------------------
    temp3 = 2 *modulus*area/h_e;  
     for A in 1:dofs_per_elem
       for B in 1:dofs_per_elem
           for q in quad_rule_values
#              //EDIT - Define Klocal.
             temp =  basis_gradient(A,myquad.quad_points[q]);
             temp2 =  basis_gradient(B,myquad.quad_points[q])*myquad.quad_weights[q];
#               // std::cout << "   temp:       "  << temp <<std::endl;
             Klocal[A,B] += temp*temp2* temp3;
            end            
        end
    end

    # ---------------------------
    #
    #      //Assemble local K and F into global K and F
    #     //You will need to used local_dof_indices[A]
    # 
    # ---------------------------
     for A in 1:dofs_per_elem   
        # add the locally generated Flocal to the F global. use the elem_dofs to provide the mapping. 
        Fglobal[elem_dofs[A]]+=Flocal[A];        
        for B in 1:dofs_per_elem # for(unsigned int B=0; B<dofs_per_elem; B++){
            #           //EDIT - add component A,B of Klocal to the correct location in K (using local_dof_indices)
            # K.add(i,j,C);*/
            Kglobal[elem_dofs[A], elem_dofs[B]]+= Klocal[A,B];
        end # }
    end#     }
    
#     println("Element $i, K = $Klocal, F = $Flocal. With dofs $elem_dofs")
    
end

LoadError: UndefVarError: dofs_per_elem not defined

# repartition the F and K matrix

In [24]:
K_fixedfixed_colum = Kglobal[:, fixed_nodes];
# F_fixedfixed = Fglobal[fixed_nodes];
disp_fixedfixed = disp[fixed_nodes];
F_fixed = K_fixedfixed_colum*disp_fixedfixed;
Fglobal -=F_fixed;

LoadError: UndefVarError: Kglobal not defined

In [25]:
# size(K_fixedfixed_colum), size(disp_fixedfixed)

In [26]:
K_freefree = Kglobal[free_nodes, free_nodes];
F_freefree = Fglobal[free_nodes];



LoadError: UndefVarError: Kglobal not defined

# calculate the solution

In [27]:
disp_freefree = inv(K_freefree)*F_freefree;


LoadError: UndefVarError: K_freefree not defined

In [28]:
# disp = zeros(Float32,num_nodes );
disp[free_nodes] += disp_freefree;
# disp[fixed_nodes]=

LoadError: UndefVarError: disp not defined

In [29]:
# disp_freefree

In [30]:
# Fglobal

In [31]:
# Kglobal

In [32]:
# plot(node_loc,Fglobal)


In [33]:
# Klocal

In [34]:
# Flocal

In [35]:
# ? collect

In [36]:
function exact(x)
   c1 = (modulus.*g2 .+f_bar .* L.^3 ./6 .- modulus .*g1) ./L;
    u =(-f_bar .* x.^3 ./6 .+c1 .*x .+modulus .*g1) ./modulus;
    return u
end

exact (generic function with 1 method)

In [37]:
y = exact(node_loc);

LoadError: UndefVarError: node_loc not defined

In [38]:
plot(node_loc, y, label = "exact")
plot!(node_loc,disp, label = "FEA")

LoadError: UndefVarError: node_loc not defined